In [18]:
# Author Emmanuel Sedicol
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import cv2
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
import sys, os

x = sys.path.append(os.path.abspath('/Users/esedicol/Desktop/Basketball-Shot-Detectection/'))
print(x)
# # Now do your import
from /Users/esedicol/Desktop/Basketball-Shot-Detectection/YOLO import yolo

# from IPython.display import clear_output

# shot_xy_coordinates = []
# player_xy_coordinates = []
# ball_xy_coordinates = []

# orangeLower = (1, 80, 110)
# orangeUpper = (8,200,175)

# CATEGORIES = ['basketball', 'hoop']
# model = tf.keras.models.load_model("/Users/esedicol/Desktop/Basketball-Shot-Detectection/MODEL")

SyntaxError: invalid syntax (<ipython-input-18-6361254f42c9>, line 15)

# Inner Functions

In [17]:
import sys
print (sys.path)

['/Users/esedicol/Desktop/Basketball-Shot-Detectection/NOTEBOOKS', '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/esedicol/Library/Python/3.7/lib/python/site-packages', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/tf_pose-0.1.1-py3.7-macosx-10.15-x86_64.egg', '/usr/local/lib/python3.7/site-packages/dill-0.2.7.1-py3.7.egg', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/IPython/extensions', '/Users/esedicol/.ipython', '/Users/esedicol/Desktop/Basketball-Shot-Detectection/config', '/Users/esedicol/Desktop/Basketball-Shot-Detectection/config', '/Users/esedicol/Desktop/Basketball-Shot-Detectection/config', '/Users/esedicol/Desktop/Basketball-Shot-Detectection/config', '/Users/

In [ ]:
# prepare the detected frame for classification
def predict_object(frame):
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (80,80))
    img = img.reshape(-1, 80, 80, 1)
    
    # predict image
    prediction = model.predict([img])
    prediction_label = CATEGORIES[int(prediction[0][0])]
    return(prediction_label)

# color detection is lower and upper hsv range
def color_detection(hsv,lower,higher):
    # create mask to detect the orange only
    mask = cv2.inRange(hsv, lower, higher)

    # find contours in the mask and initialize the current
    cnts = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

# Ball Detection Test

In [ ]:
path = "VIDEOS/BALL_DETECTION_TEST.mov"

vs = cv2.VideoCapture(path)
fps = FPS().start()

while True:
    # grab the current frame
    _,frame = vs.read()

    # break out of the application if we have no more frames to read
    if frame is None:
        break

    # resize frame so to make its constant size througout
    frame = cv2.resize(frame,(600, 400))
    detected_object = frame.copy()

    # frame filtering and image processing
    blur = cv2.GaussianBlur(frame, (15, 15),0)
    erode = cv2.erode(blur, None, iterations=2)
    dilate = cv2.dilate(erode, None, iterations=2)
    hsv = cv2.cvtColor(dilate, cv2.COLOR_BGR2HSV)

    # detect basketball using its color (i.e. orange)
    cnts = color_detection(hsv,orangeLower,orangeUpper)

    # Retrieve the largest contour in the mask, then use it to compute the minimum enclosing circle and centroid
    c = max(cnts, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(c)

    if  len(cnts) > 0:      
        # only proceed if the radius meets a minimum size
        if radius > 0 and radius < 30 :
            # Retrive the frame of detected ball
            detected_object = detected_object[int(y - 20):int(y + 20),int(x - 20):int(x + 20)] 

            # detect ball using the model I built
            if predict_object(detected_object) == "basketball" :
                
                # Append (x,y) coordinates 
                shot_xy_coordinates.append((x,y))
                
                # Draw the circle and centroid on the frame
                cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                cv2.circle(frame, (int(x), int(y)), 1, (0, 0, 255), 2)

                # Draw rectangle around the minimum enclosed circle and dipslay label of detected object
                cv2.rectangle(frame, (int(x - 20), int(y - 20)), (int(x + 20), int(y + 20)), (255,0,0), 2)
                cv2.putText(frame, "BASKETBALL", (int(x - 30), int(y - 25)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1 )

    # plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    fig.suptitle('Basketball and Player Detection')
    ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # plot at realtime the shot coordinates
    for x in range(0, len(shot_xy_coordinates)):
        ax2.scatter(shot_xy_coordinates[x][0], shot_xy_coordinates[x][1])

    # Plot Limit equals the size of the frame
    ax2.set_xlim(0, 600)
    ax2.set_ylim(0,400)
    ax2.invert_yaxis()

  
    plt.show()
    clear_output(wait=True)

fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

vs.release()

# Player Detection Test
- YOLO Detection
- Background Subtraction Dectection

### YOLO Detection Test

In [ ]:
path = "VIDEOS/PLAYER_EXTRACTION_TEST.mov"

vs = cv2.VideoCapture(path)
fps = FPS().start()

while True:
    # grab the current frame
    _,frame = vs.read()

    # break out of the application if we have no more frames to read
    if frame is None:
        break

    # resize frame so to make its constant size througout
    frame = cv2.resize(frame,(600, 450))
    detected_object = frame.copy()
    
    # detect person using YOLO
    target = "person"
    yolo.yolo_detection(target, frame)

    # retrieve player coordinate on the court using YOLO
    x_coor, y_coor = yolo.xy_personCoordinate(frame)

    # save coordinates
    player_xy_coordinates.append((int(x_coor), int(y_coor)))
    np.savetxt('/Users/esedicol/Desktop/Desktop/Basketball-Shot-Detectection/COORDINATES/person_xy_coord.txt', player_xy_coordinates, fmt='%d')
    
    # plot results
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    fig.suptitle('Basketball and Player Detection')
    ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax.axis("off")
    plt.show()
    clear_output(wait=True)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break
    

fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
vs.release()

### Detection using Background Subtraction Method

In [ ]:
def show(title, ax_one, ax_two):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    fig.suptitle(title)
    ax1.imshow(cv2.cvtColor(ax_one, cv2.COLOR_BGR2RGB))
    ax2.imshow(cv2.cvtColor(ax_two, cv2.COLOR_BGR2RGB))
    ax1.axis("off")
    plt.show()
    
    clear_output(wait=True)

vs = cv2.VideoCapture('VIDEOS/PLAYER_EXTRACTION_TEST.mov') 
fps = FPS().start()

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

while True:
    # grab the current frame
    _, frame = vs.read()
    frame = cv2.resize(frame,(600, 450))
    
    blur = cv2.GaussianBlur(frame, (15, 15),0)
    dilate = cv2.dilate(blur, None, iterations=2)
    fgmask = fgbg.apply(dilate)
    dilate = cv2.dilate(fgmask, kernel)

    (cnts, hier) = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(cnts):
        area = cv2.contourArea(contour)

        if area > 5000:
            x,y,w,h = cv2.boundingRect(contour)
    
            x_coordinate = int(x + (w/2))
            y_coordinate = int(y + h)

            cv2.rectangle(frame, (x,y), (x+w, y+h), (90,10,255), 2)
            cv2.putText(frame, "PERSON", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1 )
            cv2.circle(frame, (x_coordinate, y_coordinate), 3, (0, 0, 255), 8)

            title = "Background Sunbtraction Detection (OpenCV)"
            show(title, frame, dilate)
    
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
vs.release()